# 4.- Combinar datos númericos y categóricos con imágenes

A continuación voy a definir un modelo que sea capaz de aceptar como entrada datos numéricos y categóricos.

Crearemos una red con dos ramas, cada una de ellas encargada de trabajar con un tipo de datos. Posteriormente los resultados de ambas ramas se combinarán para obtener la predicción.

In [47]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from keras import optimizers,models,layers
from keras.applications.vgg16 import VGG16
from keras.models import Model
from keras.layers import Dropout, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import BatchNormalization, Activation
import pickle
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
import time

In [48]:
def imagenes_precio():
  X_train = pickle.load(open('Data/X_train.pickle', "rb"))
  y_train = pickle.load(open('Data/y_train.pickle', "rb"))
  X_test = pickle.load(open('Data/X_test.pickle', "rb"))
  y_test = pickle.load(open('Data/y_test.pickle', "rb"))

  #Normalización de datos
  X_train = X_train/255.0
  X_test = X_test/255.0

  return X_train, y_train, X_test, y_test

In [49]:
def df_con_imagen(path):
    
    train = pd.read_csv(path, sep = ',', decimal = '.', index_col = 0) # cargamos fichero

    #Elimino "Picture Url" que no la necesito para la parte numérica/categórica
    train.drop (['Picture Url'], axis = 1, inplace = True)

    #Normalización - tranformación logarítmica
    numerical_columns = train.columns
    numerical_columns = [i for i in numerical_columns 
                         if i not in ['Availability 30']] # Quito las que no se van a transformar

    for col in numerical_columns:
        train[col] = train[col].astype('float64').replace(0.0, 0.01) # Replacing 0s with 0.01
        train[col] = np.log(train[col])      
    
    # Separamos X and y
    X = train.drop('Price', axis = 1)
    y = train.Price    

    return X,y

In [50]:
#Defino la red. Para ello utilizo las capas del modelo seleccionado anteriormente
def create_mlp (dim):  
  modelo = models.Sequential()
  modelo.add (layers.Dense (16, input_shape = (dim,), activation = 'relu'))
  modelo.add(layers.Dense(32, activation = 'relu'))
  modelo.add(layers.Dense(32, activation = 'relu'))
  modelo.add(layers.Dense(1, activation = 'linear'))
  return modelo

In [51]:
#Creo el modelo partiendo como base del VGG16 con imagenet y reentrenando todas las capas
def create_cnn (dim):
  inputs = layers.Input (shape = dim)
  base_model = VGG16(weights = 'imagenet',
                     include_top = False, 
                     input_shape=dim)
  last = base_model.layers[-1].output
  x = Flatten()(last)  
  x = Dense(1024, name = 'fc1')(x)
  x = BatchNormalization ()(x)
  x = Activation('relu')(x)
  x = Dropout(0.5)(x)

  #Aplico otra capa densa de 512 para que coincida con el número final de neuronas de la rama Mlp
  x = Dense(512)(x)
  x = Activation('relu')(x)

  #No introduzco la última capa Dense con la función de activación Linear, esto lo haré cuando realice la combinación
  
  return Model(base_model.input, x)

## 4.1 Rama Datos numéricos

In [52]:
#Cargo los datos del dataset limpio para train
X_train, y_train = df_con_imagen('Data/train_img.csv')
#Cargo los datos del dataset limpio para test
X_test, y_test = df_con_imagen('Data/test_img.csv')

In [53]:
# Scaling
scaler = StandardScaler()
X_train = pd.DataFrame(scaler.fit_transform(X_train),
                       columns = list(X_train.columns)) 
X_test = pd.DataFrame(scaler.transform(X_test),
                      columns = list(X_test.columns))
#Separo en train y val
X_train, X_val, y_train, y_val = train_test_split(X_train,
                                                  y_train,
                                                  test_size = 0.1,
                                                  random_state = 123)
#creo el modelo
mlp = create_mlp (X_train.shape[1])

## 4.2 Rama Imágenes

In [54]:
"""primero cargo mi dataset y lo redimensiono a las dimensiones mínimas
permitidas por la VGG16 -> (48, 48, 3)"""

Xi_train, yi_train, Xi_test, yi_test = imagenes_precio()

input_shape = (48, 48, 3)

#Separo en train y val
Xi_train, Xi_val, yi_train, yi_val = train_test_split(Xi_train,
                                                      yi_train,
                                                      test_size=0.1,
                                                      random_state=123)

# redimensiono train set
X_train_resized = np.array([np.resize(img, input_shape) for img in Xi_train])

# redimensiono test set
X_test_resized = np.array([np.resize(img, input_shape) for img in Xi_test])

# redimensiono val set
X_val_resized = np.array([np.resize(img, input_shape) for img in Xi_val])

#creo el modelo
cnn = create_cnn(input_shape)

## 4.3 Combinación de ambas ramas

In [55]:
#Combino ambos modelos
combinedInput = layers.concatenate([mlp.output, cnn.output])

#Últimas capas y regresión
x = Dense(256, activation="relu")(combinedInput)
x = Dense(1, activation="linear")(x)

#Modelo final
model = Model(inputs = [mlp.input, cnn.input], outputs = x)

##  4.4. Compilación, entrenamiento y evaluación

In [56]:
def compilado(l, o, m):
    model.compile(loss = l,
                  optimizer = o,
                  metrics = [m])

In [57]:
def entrenamiento(e, b):
    model.fit([X_train, X_train_resized], y_train,
              validation_data = ([X_val, X_val_resized], y_val),
              epochs = e, batch_size = b)

In [58]:
def evaluacion():
  y_test_pred = model.predict([X_test, X_test_resized])
  y_train_pred = model.predict([X_train, X_train_resized])
  print("Training MSE:", round(mean_squared_error(y_train, y_train_pred),4))
  print("Validation MSE:", round(mean_squared_error(y_test, y_test_pred),4))
  print("\nTraining r2:", round(r2_score(y_train, y_train_pred),4))
  print("Validation r2:", round(r2_score(y_test, y_test_pred),4))

In [59]:
#Compilamos el modelo
compilado('mean_squared_error', 'adam', 'mean_squared_error')

#Entrenamos. 
entrenamiento(15, 256)

#Evaluamos el modelo
evaluacion()

Epoch 1/15
37/37 [==============================] - 20s 508ms/step - loss: 0.8439 - mean_squared_error: 0.8439 - val_loss: 565.6742 - val_mean_squared_error: 565.6742
Epoch 2/15
37/37 [==============================] - 18s 497ms/step - loss: 0.1033 - mean_squared_error: 0.1033 - val_loss: 25.4404 - val_mean_squared_error: 25.4404
Epoch 3/15
37/37 [==============================] - 18s 497ms/step - loss: 0.0509 - mean_squared_error: 0.0509 - val_loss: 0.3658 - val_mean_squared_error: 0.3658
Epoch 4/15
37/37 [==============================] - 18s 496ms/step - loss: 0.0218 - mean_squared_error: 0.0218 - val_loss: 0.1830 - val_mean_squared_error: 0.1830
Epoch 5/15
37/37 [==============================] - 18s 497ms/step - loss: 0.0167 - mean_squared_error: 0.0167 - val_loss: 0.0540 - val_mean_squared_error: 0.0540
Epoch 6/15
37/37 [==============================] - 18s 497ms/step - loss: 0.0146 - mean_squared_error: 0.0146 - val_loss: 0.0145 - val_mean_squared_error: 0.0145
Epoch 7/15
37/37

## 4.5 Más pruebas 

In [60]:
compilado('mean_squared_error', 'adam', 'mean_squared_error')
entrenamiento(15, 512)
evaluacion()

Epoch 1/15
19/19 [==============================] - 19s 934ms/step - loss: 0.0193 - mean_squared_error: 0.0193 - val_loss: 0.1554 - val_mean_squared_error: 0.1554
Epoch 2/15
19/19 [==============================] - 17s 917ms/step - loss: 0.0095 - mean_squared_error: 0.0095 - val_loss: 0.0092 - val_mean_squared_error: 0.0092
Epoch 3/15
19/19 [==============================] - 17s 918ms/step - loss: 0.0089 - mean_squared_error: 0.0089 - val_loss: 0.0238 - val_mean_squared_error: 0.0238
Epoch 4/15
19/19 [==============================] - 17s 918ms/step - loss: 0.0086 - mean_squared_error: 0.0086 - val_loss: 0.0088 - val_mean_squared_error: 0.0088
Epoch 5/15
19/19 [==============================] - 17s 916ms/step - loss: 0.0086 - mean_squared_error: 0.0086 - val_loss: 0.0107 - val_mean_squared_error: 0.0107
Epoch 6/15
19/19 [==============================] - 17s 916ms/step - loss: 0.0086 - mean_squared_error: 0.0086 - val_loss: 0.0083 - val_mean_squared_error: 0.0083
Epoch 7/15
19/19 [====

In [61]:
compilado('mean_squared_error', 'adam', 'mean_squared_error')
entrenamiento(15, 1024)
evaluacion()

Epoch 1/15
10/10 [==============================] - 19s 2s/step - loss: 0.0200 - mean_squared_error: 0.0200 - val_loss: 0.1274 - val_mean_squared_error: 0.1274
Epoch 2/15
10/10 [==============================] - 17s 2s/step - loss: 0.0096 - mean_squared_error: 0.0096 - val_loss: 0.0290 - val_mean_squared_error: 0.0290
Epoch 3/15
10/10 [==============================] - 17s 2s/step - loss: 0.0085 - mean_squared_error: 0.0085 - val_loss: 0.0100 - val_mean_squared_error: 0.0100
Epoch 4/15
10/10 [==============================] - 17s 2s/step - loss: 0.0082 - mean_squared_error: 0.0082 - val_loss: 0.0084 - val_mean_squared_error: 0.0084
Epoch 5/15
10/10 [==============================] - 17s 2s/step - loss: 0.0081 - mean_squared_error: 0.0081 - val_loss: 0.0278 - val_mean_squared_error: 0.0278
Epoch 6/15
10/10 [==============================] - 17s 2s/step - loss: 0.0080 - mean_squared_error: 0.0080 - val_loss: 0.0098 - val_mean_squared_error: 0.0098
Epoch 7/15
10/10 [======================

In [62]:
compilado('mean_squared_error', 'adam', 'mean_squared_error')
entrenamiento(15, 2048)
evaluacion()

Epoch 1/15
5/5 [==============================] - 22s 4s/step - loss: 0.0084 - mean_squared_error: 0.0084 - val_loss: 0.0344 - val_mean_squared_error: 0.0344
Epoch 2/15
5/5 [==============================] - 20s 4s/step - loss: 0.0082 - mean_squared_error: 0.0082 - val_loss: 0.0139 - val_mean_squared_error: 0.0139
Epoch 3/15
5/5 [==============================] - 20s 4s/step - loss: 0.0080 - mean_squared_error: 0.0080 - val_loss: 0.0169 - val_mean_squared_error: 0.0169
Epoch 4/15
5/5 [==============================] - 21s 4s/step - loss: 0.0080 - mean_squared_error: 0.0080 - val_loss: 0.0084 - val_mean_squared_error: 0.0084
Epoch 5/15
5/5 [==============================] - 20s 4s/step - loss: 0.0079 - mean_squared_error: 0.0079 - val_loss: 0.0089 - val_mean_squared_error: 0.0089
Epoch 6/15
5/5 [==============================] - 20s 4s/step - loss: 0.0079 - mean_squared_error: 0.0079 - val_loss: 0.0099 - val_mean_squared_error: 0.0099
Epoch 7/15
5/5 [==============================] - 20

In [63]:
compilado('mean_squared_error', 'adam', 'mean_squared_error')
entrenamiento(25, 512)
evaluacion()

Epoch 1/25
19/19 [==============================] - 20s 935ms/step - loss: 0.0081 - mean_squared_error: 0.0081 - val_loss: 0.0092 - val_mean_squared_error: 0.0092
Epoch 2/25
19/19 [==============================] - 17s 915ms/step - loss: 0.0079 - mean_squared_error: 0.0079 - val_loss: 0.0088 - val_mean_squared_error: 0.0088
Epoch 3/25
19/19 [==============================] - 17s 915ms/step - loss: 0.0079 - mean_squared_error: 0.0079 - val_loss: 0.0082 - val_mean_squared_error: 0.0082
Epoch 4/25
19/19 [==============================] - 17s 914ms/step - loss: 0.0078 - mean_squared_error: 0.0078 - val_loss: 0.0084 - val_mean_squared_error: 0.0084
Epoch 5/25
19/19 [==============================] - 17s 915ms/step - loss: 0.0077 - mean_squared_error: 0.0077 - val_loss: 0.0084 - val_mean_squared_error: 0.0084
Epoch 6/25
19/19 [==============================] - 17s 915ms/step - loss: 0.0078 - mean_squared_error: 0.0078 - val_loss: 0.0081 - val_mean_squared_error: 0.0081
Epoch 7/25
19/19 [====

In [64]:
compilado('mean_squared_error', 'adam', 'mean_squared_error')
entrenamiento(100, 512)
evaluacion()

Epoch 1/100
19/19 [==============================] - 19s 933ms/step - loss: 0.0077 - mean_squared_error: 0.0077 - val_loss: 0.0082 - val_mean_squared_error: 0.0082
Epoch 2/100
19/19 [==============================] - 17s 917ms/step - loss: 0.0076 - mean_squared_error: 0.0076 - val_loss: 0.0082 - val_mean_squared_error: 0.0082
Epoch 3/100
19/19 [==============================] - 17s 914ms/step - loss: 0.0075 - mean_squared_error: 0.0075 - val_loss: 0.0083 - val_mean_squared_error: 0.0083
Epoch 4/100
19/19 [==============================] - 17s 913ms/step - loss: 0.0075 - mean_squared_error: 0.0075 - val_loss: 0.0085 - val_mean_squared_error: 0.0085
Epoch 5/100
19/19 [==============================] - 17s 914ms/step - loss: 0.0076 - mean_squared_error: 0.0076 - val_loss: 0.0083 - val_mean_squared_error: 0.0083
Epoch 6/100
19/19 [==============================] - 17s 915ms/step - loss: 0.0075 - mean_squared_error: 0.0075 - val_loss: 0.0082 - val_mean_squared_error: 0.0082
Epoch 7/100
19/1

## 4.6 Conclusión

El mejor resultado lo hemos obtenido en la última prueba.

Este resultado es muy similar al que se consiguió sin utilizar imágenes.